In [ ]:
import getpass
import logging
from pathlib import Path
import sys
from typing import Optional, NoReturn
import time

import chromedriver_binary
import undetected_chromedriver
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains

## Constants

In [ ]:
TIMEOUT = 100

URL_LOGIN = 'https://www.tesco.com/account/login/en-GB?from=/'
EMAIL_LOGIN = 'roseannaferguson@hotmail.com'
PASSWORD_LOGIN = getpass.getpass()

## Logging

In [ ]:
file_handler = logging.FileHandler('shop-Main.log', mode='w')
file_handler.setLevel(logging.DEBUG)

console_handler = logging.StreamHandler(stream=sys.stdout)
console_handler.setLevel(logging.INFO)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=(file_handler, console_handler),
)

## Functions

In [ ]:
def wait_and_get(
    driver: webdriver.Chrome,
    value: str,
    by: Optional[By] = None,
    timeout: Optional[int] = None,
) -> WebElement:
    
    if by is None:
        by = By.XPATH
        
    if timeout is None:
        timeout = TIMEOUT
        
    logging.debug(f'{by=}, {value=}, {timeout=}')
    
    wait = expected_conditions.presence_of_element_located((by, value))
    WebDriverWait(driver, timeout).until(wait)
    
    logging.info(f'Waited for: {by=}, {value=}')
    time.sleep(1)

    return driver.find_element(by=by, value=value)
    

def wait_and_click(
    driver: webdriver.Chrome,
    value: str,
    by: Optional[By] = None,
    timeout: Optional[int] = None,
) -> NoReturn:
    element = wait_and_get(driver=driver, value=value, by=by, timeout=timeout)
    element.click()
    logging.info(f'Clicked: {value=}')
    

def wait_and_execute_click(
    driver: webdriver.Chrome,
    value: str,
    by: Optional[By] = None,
    timeout: Optional[int] = None,
) -> NoReturn:
    element = wait_and_get(driver=driver, value=value, by=by, timeout=timeout)
    driver.execute_script("arguments[0].click();", element)
    logging.info(f'Executed click via script: {value=}')
    
    
def wait_and_send_keys(
    driver: webdriver.Chrome,
    value: str,
    keys: str,
    by: Optional[By] = None,
    timeout: Optional[int] = None,
) -> NoReturn:
    element = wait_and_get(driver=driver, value=value, by=by, timeout=timeout)
    element.send_keys(keys)
    logging.info(f'Sent keys: {keys=} into {value=}')

In [ ]:
def get_driver() -> undetected_chromedriver.Chrome:
    options = undetected_chromedriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--password-store=basic')
    options.add_experimental_option(
        'prefs',
        {
            'credentials_enable_service': False,
            'profile.password_manager_enabled': False,
        },
    )

    return undetected_chromedriver.Chrome(options=options)

In [ ]:
def login() -> undetected_chromedriver.Chrome:
    driver = get_driver()
    
    driver.get(URL_LOGIN)
    
    xpath_cookies_accept = '//button[@type="submit"]//span[text()="Accept all cookies"]'
    wait_and_click(driver=driver, value=xpath_cookies_accept)

    xpath_email = '//input[@id="email"]'
    element_email = wait_and_send_keys(driver=driver, value=xpath_email, keys=EMAIL_LOGIN)

    xpath_password = '//input[@id="password"]'
    element_email = wait_and_send_keys(driver=driver, value=xpath_password, keys=PASSWORD_LOGIN)

    xpath_sign_in = '//button[@id="signin-button"]'
    element_sign_in = wait_and_click(driver=driver, value=xpath_sign_in)
    logging.info(f'Clicked {xpath_sign_in=}')